### 1. 新 01.01.2025 把 entity extraction 和  relation extraction 合并起来了, 这个代码里不产生 embeddings, 原文的 embeddings 会在step_3 的 ipynb 文件产生

In [1]:
# Cell 1: Import required libraries
# Cell 1: Import required libraries
import anthropic
import json
from pathlib import Path
from typing import Dict, List, Any
from tqdm import tqdm
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib.namespace import RDF, RDFS
import time
import os
from dotenv import load_dotenv
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np  # Added numpy import

class SimpleKnowledgeGraphBuilder:
    def __init__(self, api_key: str):
        self.client = anthropic.Anthropic(api_key=api_key)
        self.graph = Graph()
        self.SCD = Namespace("http://example.org/scd/")
        self.graph.bind("scd", self.SCD)
        
        # Simple storage for entities and relationships
        self.entities = {}  # Store entities by category
        self.relationships = []  # Store relationships

    def extract_entities_and_relationships(self, text: str) -> tuple[dict, list]:
        """Extract entities and relationships using Claude API."""
        prompt = """Given a medical text excerpt from pediatric textbooks and clinical references, perform these tasks:

        1. Extract entities in these categories:
        - Conditions
        - Symptoms
        - Treatments
        - Medications
        - Complications
        - Risk_Factors
        - Diagnostic_Tests
        - Care_Providers

        2. Identify relationships between the extracted entities using these patterns:
        - X has_symptom Y
        - X treated_by Y
        - X causes Y
        - X prevents Y
        - X diagnosed_by Y
        - X managed_by Y

        Text: {text}

        Return a JSON object with two keys:
        1. "entities": Categories as keys, arrays of strings as values
        2. "relationships": Array of objects with "subject", "predicate", "object" """

        try:
            # Add error handling for the response
            response = self.client.messages.create(
                model="claude-3-5-sonnet-20241022",
                max_tokens=4096,
                temperature=0,
                messages=[
                    {"role": "user", "content": prompt.format(text=text)}
                ]
            )
            
            # Add response validation
            if not response.content or not response.content[0].text:
                print("Empty response received")
                return {}, []
                
            try:
                result = json.loads(response.content[0].text)
                return result["entities"], result["relationships"]
            except json.JSONDecodeError as e:
                print(f"JSON parsing error: {str(e)}\nResponse: {response.content[0].text}")
                return {}, []
                
        except Exception as e:
            print(f"API error: {str(e)}")
            return {}, []

    def process_text_section(self, text: str, source_info: Dict[str, Any]):
        """Process a section of text, extracting and storing both entities and relationships."""
        entities, relationships = self.extract_entities_and_relationships(text)
        
        # Process entities
        for category, items in entities.items():
            if category not in self.entities:
                self.entities[category] = set()
            
            for item in items:
                self.entities[category].add(item)
                
                # Add to RDF graph
                entity_uri = URIRef(self.SCD[self._clean_uri(item)])
                self.graph.add((entity_uri, RDF.type, self.SCD[category]))
                self.graph.add((entity_uri, self.SCD.name, Literal(item)))
                
                # Add source information with original text
                source_uri = URIRef(self.SCD[f"source_{self._clean_uri(source_info['title'])}_{source_info['page_number']}"])
                self.graph.add((entity_uri, self.SCD.source, source_uri))
                self.graph.add((source_uri, self.SCD.title, Literal(source_info['title'])))
                self.graph.add((source_uri, self.SCD.page, Literal(source_info['page_number'])))
                self.graph.add((source_uri, self.SCD.original_text, Literal(text)))
                
                # Add metadata about the occurrence context
                occurrence_uri = URIRef(self.SCD[f"occurrence_{self._clean_uri(item)}_{self._clean_uri(source_info['title'])}_{source_info['page_number']}"])
                self.graph.add((entity_uri, self.SCD.has_occurrence, occurrence_uri))
                self.graph.add((occurrence_uri, self.SCD.in_text, Literal(text)))
                self.graph.add((occurrence_uri, self.SCD.in_source, source_uri))
        
        # Process relationships
        for rel in relationships:
            # Enhanced relationship storage with original text
            relationship_entry = {
                'subject': rel['subject'],
                'predicate': rel['predicate'],
                'object': rel['object'],
                'source': source_info,
                'original_text': text  # Adding original text to relationship metadata
            }
            self.relationships.append(relationship_entry)
            
            # Add to RDF graph
            subject_uri = URIRef(self.SCD[self._clean_uri(rel['subject'])])
            object_uri = URIRef(self.SCD[self._clean_uri(rel['object'])])
            predicate_uri = self.SCD[rel['predicate']]
            
            # Create a unique URI for this relationship instance
            rel_instance_uri = URIRef(self.SCD[f"rel_{self._clean_uri(rel['subject'])}_{self._clean_uri(rel['predicate'])}_{self._clean_uri(rel['object'])}_{source_info['page_number']}"])
            
            # Add the main relationship triple
            self.graph.add((subject_uri, predicate_uri, object_uri))
            
            # Add metadata about this relationship instance
            self.graph.add((rel_instance_uri, self.SCD.has_subject, subject_uri))
            self.graph.add((rel_instance_uri, self.SCD.has_predicate, predicate_uri))
            self.graph.add((rel_instance_uri, self.SCD.has_object, object_uri))
            self.graph.add((rel_instance_uri, self.SCD.source, source_uri))
            self.graph.add((rel_instance_uri, self.SCD.original_text, Literal(text)))

    def process_json_file(self, json_path: str) -> None:
        """Process a JSON file and update the knowledge graph."""
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            print(f"\nProcessing: {data['title']}")
            for page in tqdm(data['pages'], desc="Processing pages"):
                if page['content'].strip():
                    source_info = {
                        'title': data['title'],
                        'page_number': page['page_number']
                    }
                    
                    self.process_text_section(page['content'], source_info)
                    time.sleep(0.5)  # Rate limiting
                    
        except Exception as e:
            print(f"Error processing file {json_path}: {str(e)}")

    def save_graph(self, output_path: str):
        """Save the knowledge graph to TTL format."""
        self.graph.serialize(destination=output_path, format='turtle')

    def save_json(self, output_path: str):
        """Save entities and relationships to JSON."""
        data = {
            "entities": {category: list(entities) 
                        for category, entities in self.entities.items()},
            "relationships": self.relationships
        }
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2)

    def visualize_sample(self, n_entities=20):
        """Create a sample visualization with limited nodes for better visibility."""
        G = nx.DiGraph()
        
        # Select a sample of entities from each category
        added_entities = set()
        for category, entities in self.entities.items():
            # Take up to n_entities/len(categories) from each category
            sample_size = min(len(entities), n_entities // len(self.entities))
            sampled_entities = set(list(entities)[:sample_size])
            added_entities.update(sampled_entities)
            
            # Add nodes with category information
            for entity in sampled_entities:
                G.add_node(entity, category=category)
        
        # Add relationships for sampled entities
        for rel in self.relationships:
            if rel['subject'] in added_entities and rel['object'] in added_entities:
                G.add_edge(rel['subject'], rel['object'], 
                          relationship=rel['predicate'])
        
        # Create visualization
        plt.figure(figsize=(15, 10))
        
        # Position nodes using spring layout
        pos = nx.spring_layout(G, k=1, iterations=50)
        
        # Draw nodes with different colors for each category
        colors = plt.cm.Set3(np.linspace(0, 1, len(self.entities)))
        for i, (category, _) in enumerate(self.entities.items()):
            nx.draw_networkx_nodes(G, pos,
                                 nodelist=[n for n, d in G.nodes(data=True) 
                                         if d['category'] == category],
                                 node_color=[colors[i]],
                                 node_size=1000,
                                 alpha=0.6,
                                 label=category)
        
        # Draw edges
        nx.draw_networkx_edges(G, pos, edge_color='gray', arrows=True)
        
        # Add labels
        nx.draw_networkx_labels(G, pos, font_size=8)
        
        plt.title("Sample Knowledge Graph Visualization")
        plt.legend()
        plt.axis('off')
        return plt.gcf()

    def get_statistics(self):
        """Get basic statistics about the knowledge graph."""
        stats = {
            "Entity counts by category": {
                category: len(entities) 
                for category, entities in self.entities.items()
            },
            "Total relationships": len(self.relationships),
            "Relationship types": {}
        }
        
        # Calculate total number of entities
        stats["Total entities"] = sum(len(entities) for entities in self.entities.values())
        
        # Count relationship types
        for rel in self.relationships:
            pred = rel['predicate']
            stats["Relationship types"][pred] = stats["Relationship types"].get(pred, 0) + 1
            
        return stats

    
    def visualize_complete_graph(self):
        """Create a complete visualization showing all nodes and relationships."""
        G = nx.DiGraph()
        
        # Add all nodes with their categories
        for category, entities in self.entities.items():
            for entity in entities:
                G.add_node(entity, category=category)
        
        # Add all relationships
        for rel in self.relationships:
            G.add_edge(rel['subject'], rel['object'], 
                    relationship=rel['predicate'])
        
        # Create visualization with a larger figure size
        plt.figure(figsize=(20, 15))
        
        # Use a hierarchical layout for better organization
        pos = nx.spring_layout(G, k=2, iterations=50)
        
        # Draw nodes with different colors for each category
        colors = plt.cm.Set3(np.linspace(0, 1, len(self.entities)))
        for i, (category, entities) in enumerate(self.entities.items()):
            node_list = [n for n in G.nodes() if n in entities]
            if node_list:  # Only draw if there are nodes in this category
                nx.draw_networkx_nodes(G, pos,
                                    nodelist=node_list,
                                    node_color=[colors[i]],
                                    node_size=2000,
                                    alpha=0.6,
                                    label=category)
        
        # Draw edges with labels
        edge_labels = nx.get_edge_attributes(G, 'relationship')
        nx.draw_networkx_edges(G, pos, 
                            edge_color='gray',
                            arrows=True,
                            arrowsize=10,
                            width=0.5)
        
        # Add node labels with better formatting
        labels = {node: str(node) for node in G.nodes()}
        nx.draw_networkx_labels(G, pos, 
                            labels,
                            font_size=8)
        
        plt.title("Complete Knowledge Graph Visualization", pad=20, size=16)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.margins(x=0.2)
        plt.axis('off')
        return plt.gcf()

    def _clean_uri(self, text: str) -> str:
        """Clean text for use in URIs."""
        # Replace problematic characters
        cleaned = text.lower()
        cleaned = cleaned.replace('<', 'less_than_')
        cleaned = cleaned.replace('>', 'greater_than_')
        cleaned = cleaned.replace('%', 'percent_')
        cleaned = cleaned.replace(' ', '_')
        cleaned = cleaned.replace('-', '_')
        cleaned = cleaned.replace('.', '')
        cleaned = cleaned.replace(',', '')
        cleaned = cleaned.replace('(', '')
        cleaned = cleaned.replace(')', '')
        return cleaned

def process_directory(directory_path: str, api_key: str, graph_output_path: str, json_output_path: str):
    """Process all JSON files in a directory and build a knowledge graph."""
    # Initialize knowledge graph builder
    kg_builder = SimpleKnowledgeGraphBuilder(api_key)
    
    # Get all JSON files
    json_files = sorted(Path(directory_path).glob('*_pages.json'))
    total_files = len(json_files)
    
    print(f"Found {total_files} JSON files to process")
    
    # Process each file
    for json_file in tqdm(json_files, desc="Processing files"):
        kg_builder.process_json_file(str(json_file))
    
    # Save the results
    # kg_builder.save_graph(graph_output_path)
    kg_builder.save_json(json_output_path)
    
    print(f"\nKnowledge graph saved to: {graph_output_path}")
    print(f"Entities and relationships saved to: {json_output_path}")
    
    return kg_builder



In [5]:
# Cell 2: Main execution
if __name__ == "__main__":
    # Load environment variables
    load_dotenv()
    
    # Get API key
    api_key = os.getenv("ANTHROPIC_API_KEY")
    if not api_key:
        raise ValueError("ANTHROPIC_API_KEY environment variable is not set")
    
    # Set paths
    input_dir = r"D:\Dropbox\29. Ampelos\24_PED\PED_PITT_Aaron\backend\PDFs_Share\pdf_json_output"
    graph_output_path = Path(input_dir) / "scd_knowledge_graph.ttl"
    json_output_path = Path(input_dir) / "scd_entities_relationships.json"
    
    # Process the directory and get the builder instance
    kg_builder = process_directory(input_dir, api_key, str(graph_output_path), str(json_output_path))
    
    # Print statistics
    print("\nKnowledge Graph Statistics:")
    stats = kg_builder.get_statistics()
    for key, value in stats.items():
        print(f"\n{key}:")
        if isinstance(value, dict):
            for k, v in value.items():
                print(f"  {k}: {v}")
        else:
            print(f"  {value}")
    
    # # Create and save visualization
    # print("\nCreating sample visualization...")
    # fig = kg_builder.visualize_sample(n_entities=20)
    # plt.savefig(Path(input_dir) / "knowledge_graph_sample.png", 
    #             bbox_inches='tight', dpi=300)
    # plt.close()

Found 1 JSON files to process


Processing files:   0%|          | 0/1 [00:00<?, ?it/s]


Processing: Bright Futures Guidelines for Health... (Z-Library)


JSON parsing error: Expecting value: line 1 column 1 (char 0)
Response: For this particular text excerpt, there is limited clinical content to extract entities and relationships from, as it appears to be primarily the title page and editorial information for the "Bright Futures" guidelines. However, I can extract what's relevant:

{
    "entities": {
        "Conditions": [],
        "Symptoms": [],
        "Treatments": [],
        "Medications": [],
        "Complications": [],
        "Risk_Factors": [],
        "Diagnostic_Tests": [],
        "Care_Providers": [
            "pediatricians",
            "Health Resources and Services Administration",
            "Maternal and Child Health Bureau",
            "American Academy of Pediatrics"
        ]
    },
    "relationships": [
        {
            "subject": "American Academy of Pediatrics",
            "predicate": "manages_by",
            "object": "health supervision of infants, children, and adolescents"
        }
    ]
}


JSON parsing error: Expecting value: line 1 column 1 (char 0)
Response: For this text, I can only extract limited entities since this appears to be primarily publication/administrative information rather than medical content. Here's the structured extraction:

{
    "entities": {
        "Conditions": [],
        "Symptoms": [],
        "Treatments": [],
        "Medications": [],
        "Complications": [],
        "Risk_Factors": [],
        "Diagnostic_Tests": [],
        "Care_Providers": [
            "American Academy of Pediatrics",
            "Health Resources and Services Administration",
            "Maternal and Child Health Bureau",
            "Bright Futures National Center",
            "Division of Developmental Pediatrics and Preventive Services"
        ]
    },
    "relationships": []
}

Note: This text is primarily about organizational and publication information rather than medical content. It appears to be from the front matter or administrative section of a med

...

JSON parsing error: Extra data: line 41 column 1 (char 965)
Response: {
    "entities": {
        "Conditions": [
            "speech delay",
            "tantrums",
            "excessive tantrums",
            "behavioral problems"
        ],
        "Symptoms": [],
        "Treatments": [
            "teaching children",
            "engagement in talking"
        ],
        "Medications": [],
        "Complications": [],
        "Risk_Factors": [],
        "Diagnostic_Tests": [
            "speech delay screening"
        ],
        "Care_Providers": []
    },
    "relationships": [
        {
            "subject": "speech delay",
            "predicate": "diagnosed_by",
            "object": "speech delay screening"
        },
        {
            "subject": "speech delay",
            "predicate": "treated_by",
            "object": "engagement in talking"
        },
        {
            "subject": "behavioral problems",
            "predicate": "has_symptom",
            "obje

JSON parsing error: Extra data: line 46 column 1 (char 903)
Response: {
  "entities": {
    "Conditions": [
      "Teething"
    ],
    "Symptoms": [],
    "Treatments": [
      "Brushing teeth",
      "Temperature taking"
    ],
    "Medications": [],
    "Complications": [],
    "Risk_Factors": [
      "Television viewing",
      "Media use/exposure",
      "Tobacco use/exposure",
      "Texting"
    ],
    "Diagnostic_Tests": [],
    "Care_Providers": []
  },
  "relationships": [
    {
      "subject": "Oral health",
      "predicate": "managed_by",
      "object": "Brushing teeth"
    },
    {
      "subject": "Tobacco use/exposure",
      "predicate": "diagnosed_by",
      "object": "Prenatal Visit"
    },
    {
      "subject": "Tobacco use/exposure",
      "predicate": "diagnosed_by",
      "object": "Newborn Visit"
    },
    {
      "subject": "Tobacco use/exposure", 
      "predicate": "diagnosed_by",
      "object": "First Week Visit"
    }
  ]
}

Note: The text appears to b

JSON parsing error: Expecting value: line 1 column 1 (char 0)
Response: For this text excerpt, which appears to be from an index or table of contents, there is limited clinical content to extract meaningful entities and relationships. However, I'll structure what can be identified:

{
    "entities": {
        "Conditions": [
            "low-birth-weight"
        ],
        "Symptoms": [],
        "Treatments": [],
        "Medications": [],
        "Complications": [
            "falls"
        ],
        "Risk_Factors": [
            "Windows"
        ],
        "Diagnostic_Tests": [
            "BMI",
            "Youth Risk Behavior Surveillance System"
        ],
        "Care_Providers": []
    },
    "relationships": [
        {
            "subject": "Windows",
            "predicate": "causes",
            "object": "falls"
        }
    ]
}

Note: This text appears to be from an index or reference section, so it contains minimal clinical information to extract meaningful rel

Processing files: 100%|██████████| 1/1 [2:34:27<00:00, 9267.29s/it]



Knowledge graph saved to: D:\Dropbox\29. Ampelos\24_PED\PED_PITT_Aaron\backend\PDFs_Share\pdf_json_output\scd_knowledge_graph.ttl
Entities and relationships saved to: D:\Dropbox\29. Ampelos\24_PED\PED_PITT_Aaron\backend\PDFs_Share\pdf_json_output\scd_entities_relationships.json

Knowledge Graph Statistics:

Entity counts by category:
  Conditions: 1561
  Symptoms: 1608
  Treatments: 2324
  Medications: 159
  Complications: 776
  Risk_Factors: 2649
  Diagnostic_Tests: 777
  Care_Providers: 759
  Protective_Factors: 104
  Support_Services: 22
  Preventive_Measures: 58
  Developmental_Milestones: 31
  Nutrients: 3
  Dietary_Recommendations: 12
  Other_Entities: 8
  Safety_Devices: 6
  Programs: 6
  Supplies: 4
  Nutrition: 26
  Safety_Equipment: 21
  Foods: 16
  Nutritional_Elements: 9
  Recommended_Foods: 15
  Activities: 9
  Care_Plans: 3
  Safety_Measures: 8
  Nutritional_Items: 10
  Preventive_Equipment: 5
  Other: 9
  Developmental_Activities: 3
  Visit_Types: 15
  Preventive_Care: 